In [43]:
from definitions import data_source_path, corr_country_names, sign_level, fake_num, \
        country_col, year_col, quarter_col, month_col, date_col, incl_countries, show_results
from helper_functions_general import read_data, select_country_year_measure, month_name_to_num, rename_order_scale, \
    downsample_month_to_quarter, quarter_to_month, upsample_quarter_to_month, get_timeframe_col, get_trans, \
    get_data_path, get_fig_path, interpolate_series
from statistical_tests import stat_test
from plot_functions import plot_series

import SparseSC
import numpy as np
import pandas as pd

In [44]:
timeframe = 'm'
stat = 'stat'

source_file = 'ind_prod_1953_2023'
source_country_col = 'Country'
var_name = 'ind_prod'

In [49]:
ind_prod_m = read_data(source_path=data_source_path, file_name=source_file)
ind_prod_m

# lowercase, replace country names
ind_prod_m[source_country_col] = ind_prod_m[source_country_col].str.lower()
ind_prod_m = ind_prod_m.replace({source_country_col: corr_country_names})
    
ind_prod_m = ind_prod_m.melt(id_vars=[source_country_col],
                     value_vars=ind_prod_m.drop([source_country_col], axis=1),
                     value_name=var_name)
    
ind_prod_m[date_col] = pd.to_datetime(ind_prod_m['variable'])
ind_prod_m[year_col] = ind_prod_m[date_col].dt.year
ind_prod_m[month_col] = ind_prod_m[date_col].dt.month
ind_prod_m = ind_prod_m.drop('variable', axis=1)
    
ind_prod_m = select_country_year_measure(df=ind_prod_m, year_col=year_col, country_col=source_country_col)
ind_prod_m = ind_prod_m.replace({':': np.nan})

ind_prod_m = rename_order_scale(df=ind_prod_m, source_country_col=source_country_col,
                                var_name=var_name, var_scale=1e-2, timeframe='m')
ind_prod_m['ind_prod'] = interpolate_series(series=ind_prod_m['ind_prod'], method='median')
# infl_q = downsample_month_to_quarter(df_m=ind_prod_m, var_name=var_name, agg='mean')

ind_prod_m

,country,date,year,month,ind_prod
0,austria,2000-01-01,2000,1,0.057
1,austria,2000-02-01,2000,2,0.107
2,austria,2000-03-01,2000,3,0.097
3,austria,2000-04-01,2000,4,0.105
4,austria,2000-05-01,2000,5,0.104
...,...,...,...,...,...
4555,united_kingdom,2019-08-01,2019,8,-0.020
4556,united_kingdom,2019-09-01,2019,9,-0.017
4557,united_kingdom,2019-10-01,2019,10,-0.003
4558,united_kingdom,2019-11-01,2019,11,-0.007
